In [1]:
# 全局变量
import os
os.environ['HF_HOME'] = 'E:\PycharmProjects\huggingface'
os.environ['HF_HUB_CACHE'] = 'E:\PycharmProjects\huggingface\cache'
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [2]:
from transformers import pipeline

model_path = "models/opt-2.7b"
quant_path = "models/opt-2.7b-awq"

C:\Users\wangx\miniconda3\envs\peft\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM"}

# 加载模型
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [4]:
# 量化模型
model.quantize(tokenizer, quant_config=quant_config)

Repo card metadata block was not found. Setting CardData to empty.
AWQ: 100%|██████████| 32/32 [28:30<00:00, 53.44s/it]  


In [5]:
quant_config

{'zero_point': True, 'q_group_size': 128, 'w_bit': 4, 'version': 'GEMM'}

In [6]:
from transformers import AwqConfig, AutoConfig

# 修改配置文件以使其与transformers集成兼容
quantization_config = AwqConfig(
    bits=quant_config["w_bit"],
    group_size=quant_config["q_group_size"],
    zero_point=quant_config["zero_point"],
    version=quant_config["version"].lower(),
).to_dict()

# 预训练的transformers模型存储在model属性中，我们需要传递一个字典
model.model.config.quantization_config = quantization_config

In [7]:
# 保存模型权重
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)  # 保存分词器

('models/opt-2.7b-awq\\tokenizer_config.json',
 'models/opt-2.7b-awq\\special_tokens_map.json',
 'models/opt-2.7b-awq\\vocab.json',
 'models/opt-2.7b-awq\\merges.txt',
 'models/opt-2.7b-awq\\added_tokens.json',
 'models/opt-2.7b-awq\\tokenizer.json')